In [49]:
#Based on: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [50]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
import time

In [51]:
import numpy as np

In [52]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import string
import sklearn
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
%matplotlib inline
import seaborn as sns


In [53]:
curr_dir = os.getcwd()

In [54]:
df = pd.read_csv(curr_dir+'/data/ISIC-2017_Training_Data_metadata.csv')
df2 = pd.read_csv(curr_dir+'/data/ISIC-2017_Training_Part3_GroundTruth.csv')

In [55]:
 df = df.merge(df2, left_on='image_id', right_on='image_id', how='outer')

In [56]:
df.head(5)

,image_id,age_approximate,sex,melanoma,seborrheic_keratosis
0,ISIC_0000000,55,female,0.0,0.0
1,ISIC_0000001,30,female,0.0,0.0
2,ISIC_0000002,60,female,1.0,0.0
3,ISIC_0000003,30,male,0.0,0.0
4,ISIC_0000004,80,male,1.0,0.0


In [57]:
dict_df = {'age_max_min':[max([x  for x in df.age_approximate if x!='unknown']),min([x  for x in df.age_approximate if x!='unknown'])],
           'age_groups':set(df.age_approximate),
           'gender':set(df.sex),
          }

In [58]:
#dict_df.get('max_min_age')[0]=filter(lambda x: max(x), df.age_approximate)
set(df.age_approximate)
for x in df.age_approximate:
    if not x:
        print(x)

In [59]:
print('Number of unknown age: ', sum([1  for x in df.age_approximate if x=='unknown']))

Number of unknown age:  256


In [60]:
print('Number of unknown sex: ', sum([1  for x in df.sex if x=='unknown']))

Number of unknown sex:  229


In [61]:
dict_df['age_max_min']

['85', '10']

In [62]:
dict_df['age_groups']

{'10',
 '15',
 '20',
 '25',
 '30',
 '35',
 '40',
 '45',
 '5',
 '50',
 '55',
 '60',
 '65',
 '70',
 '75',
 '80',
 '85',
 'unknown'}

In [63]:
dict_df['gender']

{'female', 'male', 'unknown'}

In [64]:
df.columns

Index(['image_id', 'age_approximate', 'sex', 'melanoma',
       'seborrheic_keratosis'],
      dtype='object')

----

### Random Forest

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [66]:
columns=['age_approximate', 'sex']
X= df[columns]  ##this makes a copy
y=df[['melanoma']]
X.head(2)

,age_approximate,sex
0,55,female
1,30,female


In [67]:
X.sex = X.sex.map({'female': 0, 'male': 1,'unknown':2}) #need to change all the values for this

In [68]:
#X.age_approximate = X.age_approximate.map({'unknown': '0'})
X.loc[X.age_approximate == 'unknown', 'age_approximate'] = 0

In [69]:
dict_df

{'age_groups': {'10',
  '15',
  '20',
  '25',
  '30',
  '35',
  '40',
  '45',
  '5',
  '50',
  '55',
  '60',
  '65',
  '70',
  '75',
  '80',
  '85',
  'unknown'},
 'age_max_min': ['85', '10'],
 'gender': {'female', 'male', 'unknown'}}

In [80]:
#X, y = make_classification(n_samples=1000, n_features=4,
#                           n_informative=2, n_redundant=0,
#                           random_state=0, shuffle=False)
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
clf.fit(X, y)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
print(clf.feature_importances_)
print(clf.predict([[7, 1]]))

[0.75879854 0.24120146]
[0.]


In [82]:
t = sklearn.tree.export_graphviz(clf.estimators_,feature_names = X.columns,
                class_names = X.columns,
                rounded = True, proportion = False, 
                precision = 1, filled = True) 

print(t)

TypeError: [DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False,
            random_state=209652396, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False,
            random_state=398764591, splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_le